##Green Taxi Data

Extract, clean, transform and load Green Taxi trip data for a month.

Create fact, load as an unmanaged table, as well as register as a global temp view.

In [0]:

dbutils.widgets.text("ProcessMonth", "201901", "Process Month (yyyymm)")

In [0]:

paraMonth = dbutils.widgets.get("ProcessMonth")

In [0]:
print("Starting to extract Green Taxi data")

In [0]:
filepath="/mnt/taxidata1/GreenTaxiTripData_"+paraMonth+".csv"
#print(filepath)

In [0]:
greentaxidf = spark.read.format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", "\t") \
  .load(filepath)


In [0]:
print("Before = " ,greentaxidf.count())
greentaxidf=greentaxidf.filter("trip_distance >0.0") \
  .filter(greentaxidf. passenger_count >0.0) \
  .na.drop(how="any",subset=["PULocationID","DOLocationID"])\
  .na.fill('5','payment_type')\
  .na.fill('1','RatecodeID')\
  .filter((greentaxidf.lpep_pickup_datetime >'2018-12-01' ) & (greentaxidf.lpep_pickup_datetime <'2019-01-01'))
print("After = " ,greentaxidf.count())

In [0]:
print("Starting transformation on Green Taxi data")
from pyspark.sql.functions import *
greentaxidf = greentaxidf.select(\
  "VendorID",\
  col("passenger_count").alias("PassengerCount"),\
  col("trip_distance").alias("TripDistance"),\
  col("lpep_pickup_datetime").alias("PickupTime"),\
  col("lpep_dropoff_datetime").alias("DropTime"), \
  col("PUlocationID").alias("PickupLocationId"), \
  col("DOlocationID").alias("DropLocationId"), \
  "RatecodeID", \
  col("total_amount").alias("TotalAmount"),\
  col("payment_type").alias("PaymentType")\
  )\
  .withColumn("TripYear", year(col("PickupTime")))\
  .withColumn("TripMonth", month("PickupTime"))\
  .withColumn("TripDay", dayofmonth("PickupTime"))\
  .withColumn("TripTimeInMinutes",round(\
  (unix_timestamp("DropTime") - unix_timestamp("PickupTime"))/60))\
  .withColumn("TripType", 
    when(greentaxidf.RatecodeID == 6,"SharedTrip")\
    .otherwise("SoloTrip")  )\
    .drop("RatecodeID")         
                    
print("Applied transformations on Green Taxi data")

In [0]:

greentaxidf.createOrReplaceGlobalTempView("FactGreenTaxiTripData")

print("Saved Green Taxi fact as a global temp view")

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS taxiservicedb1

In [0]:

print("Starting to save Green Taxi dataframe as a fact and unmanaged table")

greentaxidf.write \
  .mode("Overwrite") \
  .option("path","/mnt/taxidata/destination/GreenTaxiData.parquet") \
  .saveAsTable("taxiservicedb1.FactGreenTaxiTripData")


print("Saved Green Taxi dataframe as a fact and unmanaged table")

In [0]:
dbutils.notebook.exit("Success")